# Classification Model

**Purpose of script:**

- Test regression model suitability in data fusion context
- Test different regressors

## Data Prep

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor

In [3]:
df_path = r"../Data/combined/"
df = pd.read_parquet(df_path + 'melt_2019-07-01_extended.parquet.gzip')
df = df.fillna(-1)

In [4]:
X = df[['x', 'y', 'mw_value', 'col', 'row', 'v1', 'v2', 'v3', 'v4', 'v6', 'v7', 'v8', 'v9', 'mean']] # v5 is duplicated
y = df[['opt_value']]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training, 20% test and 10% validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, random_state=1)

In [5]:
y_train_binary = y_train["opt_value"].apply(lambda x: 1 if x >= 0.64 else 0)
y_test_binary = y_test["opt_value"].apply(lambda x: 1 if x >= 0.64 else 0)
y_val_binary = y_val["opt_value"].apply(lambda x: 1 if x >= 0.64 else 0)

In [6]:
y_train_buckets = y_train.copy()
y_train_buckets['binned_opt_value'] = pd.cut(y_train_buckets['opt_value'], 
                                    list(np.arange(0, 0.41, 0.2)) 
                                    + [0.64] 
                                    + list(np.arange(0.8, 2.01, 0.2)) 
                                    + [7.0])

buckets = list(y_train_buckets['binned_opt_value'].unique())
buckets.sort()
num_buckets = len(buckets)
value_bucket_lookup = dict(zip(buckets, range(num_buckets)))
y_train_buckets['binned_opt_value_code'] = y_train_buckets['binned_opt_value'].replace(value_bucket_lookup).values

In [7]:
# same for test
y_test_buckets = y_test.copy()
y_test_buckets['binned_opt_value'] = pd.cut(y_test_buckets['opt_value'], 
                                    list(np.arange(0, 0.41, 0.2)) 
                                    + [0.64] 
                                    + list(np.arange(0.8, 2.01, 0.2)) 
                                    + [7.0])

buckets_test = list(y_test_buckets['binned_opt_value'].unique())
buckets_test.sort()
num_buckets_test = len(buckets_test)
value_bucket_lookup_test = dict(zip(buckets_test, range(num_buckets_test)))
y_test_buckets['binned_opt_value_code'] = y_test_buckets['binned_opt_value'].replace(value_bucket_lookup_test).values

In [8]:
y_val_buckets = y_val.copy()
y_val_buckets['binned_opt_value'] = pd.cut(y_val_buckets['opt_value'], 
                                    list(np.arange(0, 0.41, 0.2)) 
                                    + [0.64] 
                                    + list(np.arange(0.8, 2.01, 0.2)) 
                                    + [7.0])

buckets_val = list(y_val_buckets['binned_opt_value'].unique())
buckets_val.sort()
num_buckets_val = len(buckets_val)
value_bucket_lookup_val = dict(zip(buckets_val, range(num_buckets_val)))
y_val_buckets['binned_opt_value_code'] = y_val_buckets['binned_opt_value'].replace(value_bucket_lookup_val).values

## Decision Tree Classifier

### Hyperparameter Tuning - done

In [9]:
params = {
    'max_depth': list(range(10, 30, 2)) + [None],
    'min_samples_leaf': [1, 5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

classifier = DecisionTreeClassifier(random_state=0)
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [10]:
%%time
grid_search.fit(X_train, y_train_binary)
grid_search.best_params_

Fitting 4 folds for each of 132 candidates, totalling 528 fits
CPU times: total: 10.3 s
Wall time: 7min 41s


{'criterion': 'gini', 'max_depth': 22, 'min_samples_leaf': 1}

In [14]:
%%time
grid_search.fit(X_train, y_train_buckets['binned_opt_value_code'])
grid_search.best_params_

Fitting 4 folds for each of 132 candidates, totalling 528 fits
CPU times: total: 15.3 s
Wall time: 8min 45s


{'criterion': 'entropy', 'max_depth': 22, 'min_samples_leaf': 5}

### Binary Classification

In [11]:
classifier = DecisionTreeClassifier(max_depth=22, criterion="gini", min_samples_leaf=1, random_state=0)
classifier.fit(X_train, y_train_binary)

y_predicted = classifier.predict(X_test)

In [12]:
rmse = np.sqrt(mean_squared_error(y_test_binary, y_predicted))
rmse

0.1705192940362685

In [13]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.9709231703613727

### Multiclass (buckets) classification

In [15]:
classifier = DecisionTreeClassifier(criterion='entropy', max_depth=22, min_samples_leaf=5, random_state=0)
classifier.fit(X_train, y_train_buckets['binned_opt_value_code'])

y_predicted = classifier.predict(X_test)

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_buckets["binned_opt_value_code"], y_predicted))
rmse

0.7995927827845888

In [17]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.2919640145763741

## Random Forest Classifier - BEST

### Hyperparameter Tuning - undone

In [37]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

params = {
    'max_depth': list(range(10, 30, 2)) + [None],
    'min_samples_leaf': [1, 5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

classifier = RandomForestClassifier(random_state=0)
grid_search = RandomizedSearchCV(estimator=classifier, 
                                param_distributions=params,
                                cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [38]:
%%time
grid_search.fit(X_train, y_train_binary)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


KeyboardInterrupt: 

In [36]:
grid_search.best_params_

{'min_samples_leaf': 1, 'max_depth': None, 'criterion': 'entropy'}

### Binary Classification - BEST

In [9]:
classifier = RandomForestClassifier(random_state=0)
classifier.fit(X_train, y_train_binary)

y_predicted = classifier.predict(X_test)

In [174]:
rmse = np.sqrt(mean_squared_error(y_test_binary, y_predicted))
rmse

0.15735984771292208

In [175]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.975237878327766

### Multiclass (buckets) Classification

In [178]:
classifier = RandomForestClassifier(random_state=0)
classifier.fit(X_train, y_train_buckets['binned_opt_value_code'])

y_predicted = classifier.predict(X_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [180]:
rmse = np.sqrt(mean_squared_error(y_test_buckets["binned_opt_value_code"], y_predicted))
rmse

0.7264454196061586

In [181]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.28989523231096265

## Logistic Regression

### Binary Classification

In [184]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train_binary)

y_predicted = classifier.predict(X_test)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [186]:
rmse = np.sqrt(mean_squared_error(y_test_binary, y_predicted))
rmse

0.4755445038683857

In [187]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.773857424840571

### Multiclass (buckets) Classification

In [193]:
classifier = LogisticRegression(random_state=0, max_iter=500)
classifier.fit(X_train, y_train_buckets['binned_opt_value_code'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [194]:
y_predicted = classifier.predict(X_test)

In [195]:
rmse = np.sqrt(mean_squared_error(y_test_binary, y_predicted))
rmse

0.7612984709552048

In [196]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.42042463812126735

## k-nearest Neighbors Classification

### Binary Classification

In [ ]:
parameters = {"n_neighbors": range(1, 10)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train_binary)
gridsearch.best_params_

In [207]:
classifier = KNeighborsRegressor(n_neighbors=5)
classifier.fit(X_train, y_train_binary)

y_predicted = classifier.predict(X_test).astype(int)

In [208]:
rmse = np.sqrt(mean_squared_error(y_test_binary, y_predicted))
rmse

0.24455038163703965

In [209]:
accuracy = accuracy_score(y_test_binary, y_predicted)
accuracy

0.9401951108411782

### Multiclass (buckets) Classification

In [ ]:
parameters = {"n_neighbors": range(1, 10)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train_buckets['binned_opt_value_code'])
gridsearch.best_params_

In [ ]:
classifier = KNeighborsRegressor(n_neighbors=4)
classifier.fit(X_train, y_train_buckets['binned_opt_value_code'])

y_predicted = classifier.predict(X_test).astype(int)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test_buckets['binned_opt_value_code'], y_predicted))
rmse

In [ ]:
accuracy = accuracy_score(y_test_buckets['binned_opt_value_code'], y_predicted)
accuracy